In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

Using TensorFlow backend.


In [6]:
def process_image(img):
    image = cv2.resize(img, (416, 416), cv2.INTER_AREA)
    image = np.array(image)
    image = image / 255.
    image = np.expand_dims(image, axis = 0)
    
    return image
#returns (1, 416, 416, 3)

In [3]:
def get_classes(file):
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    
    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box
        
        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))  #adjust to avoid too much of an overlap
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
        
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)
        
        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))
        
    print("\n")    
        

In [7]:
def detect_image(image, yolo, all_classes):
    
    pimage = process_image(image)
    
    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()
    
    print("Time: {0:.2f}s".format(end - start))
    
    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)
        
    return image    

In [8]:
yolo = YOLO(0.6, 0.5)  #first is the object threshold and the second is the box threshold

file = 'data/coco_classes.txt'
all_classes = get_classes(file)

/home/smj/miniconda3/envs/python-cvcourse/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [16]:
f = 'person.jpg'
path = 'images/test/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

Time: 3.49s
class: person, score: 1.00
box coordinate x,y,w,h: [187.31460571  80.93353724  92.65534401 309.94415426]
class: horse, score: 1.00
box coordinate x,y,w,h: [397.24105835 136.86003971 215.32857895 208.7956531 ]
class: dog, score: 1.00
box coordinate x,y,w,h: [ 61.95981026 263.09572697 144.4931221   88.55083156]




True